# 파트 04
----
인구이동 데이터의 2차 전처리 과정 행하기 전에 Dask를 이용하여 일괄적인 방법으로 데이터의 온전성을 확인한다. 누락 데이터는 없는지를 확인하고 비세대주 중에 성별이나 나이가 기입된 경우는 없는지 등을 확인한다. 아래는 데이터 탐구 과정에서 얻은 몇가지 수치이다.

전입과 전출이 다른 경우: 3508만개

전입과 전출이 같은 경우: 2791만개

비세대주의 경우: 1795만개

In [1]:
%matplotlib inline
import glob
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from __future__ import division
from __future__ import print_function

In [2]:
path = 'raw_data\\'

In [3]:
df_population = dd.read_csv(path + '03_pop_*.csv', encoding='utf-8')

In [4]:
pbar = ProgressBar()
pbar.register()

In [5]:
# see if we are missing any data
df_population.count().compute()

[########################################] | 100% Completed | 20.3s


전입_년      63000200
전입_월      63000200
사유        63000200
세대주_관계    63000200
세대주_나이    63000200
세대주_성별    63000200
세대_관련     63000200
이동_총인구    63000200
이동_남인구    63000200
이동_여인구    63000200
전입코드      63000200
전출코드      63000200
dtype: int64

In [6]:
df_population.head()

[########################################] | 100% Completed |  1.4s


,전입_년,전입_월,사유,세대주_관계,세대주_나이,세대주_성별,세대_관련,이동_총인구,이동_남인구,이동_여인구,전입코드,전출코드
0,2006,1,1,1,41,2,1,3,1,2,11110,11440
1,2006,1,1,1,34,1,2,1,1,0,11110,41285
2,2006,1,1,1,49,1,1,4,3,1,11110,11290
3,2006,1,1,0,-1,-1,2,1,0,1,11110,41480
4,2006,1,1,1,45,2,1,2,0,2,11110,11380


In [7]:
# performing some exploratory data analysis
df_population.loc[df_population.loc[:, u'전입코드'] != df_population.loc[:, u'전출코드']].compute().shape

[########################################] | 100% Completed | 21.8s


(35083111, 12)

In [8]:
df_population.loc[df_population.loc[:, u'전입코드'] == df_population.loc[:, u'전출코드']].compute().shape

[########################################] | 100% Completed | 21.8s


(27917089, 12)

In [9]:
df_population[u'세대주_관계'].value_counts().compute()

[########################################] | 100% Completed | 19.7s


1    45047545
0    17952655
Name: 세대주_관계, dtype: int64

In [10]:
# performing sanity check to see if there are any non-householder data with
# householder age or householder gender specified
df_population.loc[(df_population.loc[:, u'세대주_관계'] == 0) & (df_population.loc[:, u'세대주_나이'] >= 0)].compute()

[########################################] | 100% Completed | 19.1s


,전입_년,전입_월,사유,세대주_관계,세대주_나이,세대주_성별,세대_관련,이동_총인구,이동_남인구,이동_여인구,전입코드,전출코드


In [11]:
df_population.loc[(df_population.loc[:, u'세대주_관계'] == 0) & (df_population.loc[:, u'세대주_성별'] >= 0)].compute()

[########################################] | 100% Completed | 19.1s


,전입_년,전입_월,사유,세대주_관계,세대주_나이,세대주_성별,세대_관련,이동_총인구,이동_남인구,이동_여인구,전입코드,전출코드
